## Two phase flow.
### pezoflow equation.

$$\frac{\partial \rho_w S_w m}{\partial t} + \frac{\partial}{\partial x}(\rho_w W_w)=0$$
$$\frac{\partial \rho_o S_o m}{\partial t} + \frac{\partial}{\partial x}(\rho_o W_o)=0$$
$$W_w=-\frac{k_{rw} k}{\mu_w}\frac{\partial p}{\partial x}$$
$$W_o=-\frac{k_{ro} k}{\mu_o}\frac{\partial p}{\partial x}$$
$$S_o+S_w=1$$
$$m(p)=m_0+\beta_r*(p-p_0)$$
$$\rho_{w,o}(p) = \rho_{w0, o0}(1 + \beta_{w,o}*(p-p_0))$$

$$x\in[0,L]$$
$$t\in[0,T]$$

*Initial and boundary conditions*
$$p(x, t=0)=p_0$$
$$S_w(x, t=0)=S_{w0}$$
$$W_w(x=0, t)=\frac{Q}{S}$$
$$S_w(x=0, t)=1$$
$$p(x=L, t)=p_0$$

1.
$$\rho_w m \frac{\partial S_w}{\partial t} + \rho_w S_w \frac{\partial m}{\partial t} + m S_w \frac{\partial \rho_w}{\partial t} - \frac{\partial}{\partial x}(\rho_w\frac{k_{rw} k}{\mu_w}\frac{\partial p}{\partial x})=0 $$
$$\rho_o m \frac{\partial S_o}{\partial t} + \rho_w S_o \frac{\partial m}{\partial t} + m S_o \frac{\partial \rho_o}{\partial t} - \frac{\partial}{\partial x}(\rho_w\frac{k_{ro} k}{\mu_o}\frac{\partial p}{\partial x})=0 $$

2.
$$ m \frac{\partial S_w}{\partial t} + S_w \beta_r \frac{\partial p}{\partial t} + m S_w \frac{\rho_{w0}}{\rho_w} \beta_w\frac{\partial p}{\partial t} - \frac{1}{\rho_w}\frac{\partial}{\partial x}(\rho_w\frac{k_{rw} k}{\mu_w}\frac{\partial p}{\partial x})=0 $$
$$m \frac{\partial S_o}{\partial t} +  S_o \beta_r \frac{\partial p}{\partial t} + m S_o \frac{\rho_{o0}}{\rho_o} \beta_o\frac{\partial p}{\partial t} - \frac{1}{\rho_o}\frac{\partial}{\partial x}(\rho_o\frac{k_{ro} k}{\mu_o}\frac{\partial p}{\partial x})=0 $$


3. summarize
$$\frac{\partial p}{\partial t}(S_w \beta_r+(1-S_w) \beta_r + m S_w \frac{\rho_{w0}}{\rho_w} \beta_w+m (1-S_w) \frac{\rho_{o0}}{\rho_o} \beta_o) - \frac{1}{\rho_w}\frac{\partial}{\partial x}(k(\frac{k_{rw} \rho_w}{\mu_w})\frac{\partial p}{\partial x})-\frac{1}{\rho_o}\frac{\partial}{\partial x}(k(\frac{k_{ro} \rho_o}{\mu_o})\frac{\partial p}{\partial x})=0 $$
$$ m \frac{\partial S_w}{\partial t} + S_w (\beta_r + m \frac{\rho_{w0}}{\rho_w} \beta_w) \frac{\partial p}{\partial t} - \frac{1}{\rho_w}\frac{\partial}{\partial x}(\rho_w\frac{k_{rw} k}{\mu_w}\frac{\partial p}{\partial x})=0 $$


3. final
$$\frac{\partial p}{\partial t}(\beta_r + m S_w \frac{\rho_{w0}}{\rho_w} \beta_w+m (1-S_w) \frac{\rho_{o0}}{\rho_o} \beta_o) - \frac{1}{\rho_w}\frac{\partial}{\partial x}(k(\frac{k_{rw} \rho_w}{\mu_w})\frac{\partial p}{\partial x})-\frac{1}{\rho_o}\frac{\partial}{\partial x}(k(\frac{k_{ro} \rho_o}{\mu_o})\frac{\partial p}{\partial x})=0 $$
$$ m \frac{\partial S_w}{\partial t} + S_w (\beta_r + m \frac{\rho_{w0}}{\rho_w} \beta_w) \frac{\partial p}{\partial t} - \frac{1}{\rho_w}\frac{\partial}{\partial x}(\rho_w\frac{k_{rw} k}{\mu_w}\frac{\partial p}{\partial x})=0 $$


4. non dimensional

$$\overline{p} = \frac{p}{p_0}, \overline{t} = \frac{t}{T}, \overline{x} = \frac{x}{L}$$

$$\frac{1}{\beta_r}\frac{\partial \overline{p}}{\partial \overline{t}}(\beta_r + m S_w \frac{\rho_{w0}}{\rho_w} \beta_w+m (1-S_w) \frac{\rho_{o0}}{\rho_o} \beta_o)

- \frac{T}{\rho_w L^2 \beta_r}\frac{\partial}{\partial \overline{x}}(k(\frac{k_{rw} \rho_w}{\mu_w})\frac{\partial \overline{p}}{\partial \overline{p}})

-\frac{T}{\rho_o L^2 \beta_r}\frac{\partial}{\partial \overline{x}}(k(\frac{k_{ro} \rho_o}{\mu_o})\frac{\partial \overline{p}}{\partial \overline{x}})=0 $$
$$ m \frac{\partial S_w}{\partial \overline{t}}

+ S_w (\beta_r + m \frac{\rho_{w0}}{\rho_w} \beta_w) \frac{\partial \overline{p}}{\partial \overline{t}}p_0

- \frac{p_0 T}{\rho_w L^2}\frac{\partial}{\partial \overline{x}}(\rho_w\frac{k_{rw} k}{\mu_w}\frac{\partial \overline{p}}{\partial \overline{x}})=0 $$

5. bndry cond

$$\frac{k_{rw} k p_0}{\mu_w L}\frac{\partial \overline{p}}{\partial \overline{x}} = \frac{Q}{S}$$
$$\overline{p}=1$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.optimize import newton_krylov, fsolve, least_squares, root
from scipy.interpolate import interp1d
from sol_plot import solution_plot
from tqdm.notebook import tqdm


def expit(x):
    try:
        return 1 / (1 + np.exp(-x))
    except:
        return 0.

Swi = 0.05
Sor = 0.2

k_water = lambda x: np.where(x > Swi, (x - Swi)**2, 0)
k_oil = lambda x: np.where(x < 1 - Sor, (1 - x - Sor)**2, 0)

dk_water = lambda x: np.where(x > Swi, 2*(x-Swi), 0) 
dk_oil = lambda x: np.where(x < 1 - Sor, -2*(1-x-Sor), 0)

plt.plot(np.linspace(0,1,100), k_water(np.linspace(0,1,100)))
plt.plot(np.linspace(0,1,100), k_oil(np.linspace(0,1,100)))

In [6]:
class TwoPhase():
    def __init__(self,
                 mu_h,
                 mu_o,
                 glad,
                 G,
                 k,
                 m0,
                 beta_r,
                 rho_w0,
                 beta_w,
                 rho_o0,
                 beta_o,
                 t_0,
                 T,
                 nt,
                 tt,
                 pow_n,
                 x_0,
                 L,
                 S,
                 nx,
                 p_0,
                 s_0,
                 s_left,
                 s_right,
                 mu_type_o,
                 mu_type_w,
                 p_left=None,
                 p_right=None,
                 Q=None) -> None:
        self.mu_h = mu_h
        self.mu_o = mu_o
        self.glad = glad
        self.G = G
        self.k = k
        self.Q = Q
        self.m0 = m0
        self.beta_r = beta_r
        self.rho_w0 = rho_w0
        self.beta_w = beta_w
        self.rho_o0 = rho_o0
        self.beta_o = beta_o
        self.t_0 = t_0
        self.T = T
        self.nt = nt
        self.tt = tt
        self.pow_n = pow_n
        self.x_0 = x_0
        self.L = L
        self.nx = nx
        self.p_0 = p_0
        self.s_0 = s_0
        self.s_left = s_left
        self.s_right = s_right
        self.time = np.linspace(t_0, T, int(nt), dtype=np.float64)/self.T
        self.dt = self.time[1]-self.time[0]
        self.tt = self.time.flat[np.abs(self.time - tt).argmin()]
        self.x = np.linspace(0, L, nx, dtype=np.float64)/self.L
        self.dx = self.x[1]-self.x[0]
        self.mu_oil_arr = []
        self.mu_water_arr = []
        self.grad_p = []
        self.grad_mu_o = []
        self.grad_mu_w = []
        self.mu_stop_o = np.zeros_like(self.x)
        self.mu_stop_w = np.zeros_like(self.x)
        if mu_type_o not in ["mu_2side", "mu_run", "mu"]:
            raise NameError
        self.mu_type_o = mu_type_o
        self.mu_type_w = mu_type_w
        self.p_left = p_left
        self.p_right = p_right
        self.S = S

    def m(self, p):
        return self.m0 + self.beta_r*(p*self.p_0-self.p_0)

    def rho_w(self, p):
        return self.rho_w0*(1+self.beta_w*(p*self.p_0-self.p_0))

    def rho_o(self, p):
        return self.rho_o0*(1+self.beta_o*(p*self.p_0-self.p_0))
    
    def mu_water(self, grad, grad_old, mu_stop_w):
        grad = abs(grad)*self.p_0/self.L
        grad_old = abs(grad_old)*self.p_0/self.L
        mu_h_w = self.mu_h*1.5
        mu_o_w = self.mu_o/1.5
        G_w = self.G/1.5
        # print(np.max(grad), G_w)
        mu = (mu_h_w-mu_o_w) * expit (self.glad * (-grad + G_w)) + mu_o_w
        if self.mu_type_w=='mu_2side':
            mu_new = (mu_h_w-mu_o_w) * expit (self.glad*1.1 * (-grad + G_w)) + mu_o_w
            res = np.where(mu<=mu_o_w, 1, 0)
            res = np.where(mu_stop_w+res>=1, 1, 0)
            result = np.where(res==1, mu_new, mu)
        elif self.mu_type_w=='mu_run':
            result = mu
            res = self.mu_stop_w
        else:
            res = np.where(grad>grad_old, grad, grad_old)
            result = (mu_h_w-mu_o_w) * expit (self.glad * (-res + G_w)) + mu_o_w
        return result, res

    def mu_oil(self, grad, grad_old, mu_stop_o):
        grad = abs(grad)*self.p_0/self.L
        grad_old = abs(grad_old)*self.p_0/self.L
        mu = (self.mu_h-self.mu_o) * expit (self.glad * (-grad + self.G)) + self.mu_o
        if self.mu_type_o=='mu_2side':
            mu_new = (self.mu_h/1. - self.mu_o) * expit (self.glad * (-grad + self.G/100)) + self.mu_o
            res = np.where(mu<=self.mu_o*1., 1, 0)
            res = np.where(mu_stop_o+res>=1, 1, 0)
            result = np.where(res==1, mu_new, mu)
        elif self.mu_type_o=='mu_run':
            result = mu
            res = None
        else:
            res = np.where(grad>grad_old, grad, grad_old)
            result = (self.mu_h-self.mu_o) * expit (self.glad * (-res + self.G)) + self.mu_o
        return result, res

    def rate(self, t):
        n = self.pow_n
        a = self.Q*(self.T-self.t_0) / (self.tt**(n+1)/(n+1) + self.tt**n*(self.T-self.tt))
        return np.where(t<self.tt,a*t**(n),a*self.tt**(n))

    def lam_w(self, s, p, grad, grad_old, mu):
        return self.k*k_water(s)*self.rho_w(p) / self.mu_water(grad, grad_old, mu)[0]

    def lam_o(self, s, p, grad, grad_old, mu):
        return self.k*k_oil(s)*self.rho_o(p) / self.mu_oil(grad, grad_old, mu)[0]

    def coef_dp_dt(self, p, s):
        return self.beta_r + self.m(p)*s*self.beta_w*self.rho_w0/self.rho_w(p)+\
                   self.m(p)*(1-s)*self.beta_o*self.rho_o0/self.rho_o(p)

    def solution_init(self):
        p = np.zeros((self.nt, self.nx), dtype=np.float64)
        p[0, :] = self.p_0/self.p_0

        s = np.zeros((self.nt, self.nx), dtype=np.float64)
        s[0,:] = self.s_0
        s[:,0] = self.s_left
        return p, s

    def grad(self, p, dx, axis='center'):
        res = np.ones_like(p)
        if axis=='left':
            res[1:] = (p[1:]-p[:-1])/dx
            res[0] = (p[1]-p[0])/dx
        elif axis=='right':
            res[:-1] = (p[1:]-p[:-1])/dx
            res[-1] = (p[-1]-p[-2])/dx
        else:
            res = np.gradient(p, dx)
        return res

    def residual(self, u_new, u_old, s_old, t_step, dt, dx):
        """Выражение невязки для метода Ньютона-Крылова."""
        N = len(u_new)
        res = np.zeros(N, dtype=np.float64)
        grad = self.grad(u_new, self.dx)
        grad_old = self.grad_p[-1]
        k_left_w = self.lam_w(s_old[1:-1], u_new[1:-1], grad[1:-1], grad_old[1:-1], self.mu_stop_w[1:-1])
        k_right_w = self.lam_w(s_old[2:], u_new[2:], grad[2:], grad_old[2:], self.mu_stop_w[2:])

        k_left_o = self.lam_o(s_old[1:-1], u_new[1:-1], grad[1:-1], grad_old[1:-1], self.mu_stop_o[1:-1])
        k_right_o = self.lam_o(s_old[2:], u_new[2:], grad[2:], grad_old[2:], self.mu_stop_o[2:])


        res[1:-1] = 1*self.coef_dp_dt(u_new[1:-1], s_old[1:-1])*(u_new[1:-1] - u_old[1:-1]) / dt - \
                    self.T/(self.rho_w(u_new[1:-1])*self.L**2)*(k_right_w*(u_new[2:] - u_new[1:-1]) / dx**2 - k_left_w*(u_new[1:-1] - u_new[:-2]) / dx**2) -\
                    self.T/(self.rho_o(u_new[1:-1])*self.L**2)*(k_right_o*(u_new[2:] - u_new[1:-1]) / dx**2 - k_left_o*(u_new[1:-1] - u_new[:-2]) / dx**2)

        # Граничные условия
        if self.p_left is not None:
            res[0] = u_new[0] - self.p_left/self.p_0
        else:
            res[0] = self.p_0/self.L*(u_new[1] - u_new[0]) / dx * k_water(s_old[0])* self.k / self.mu_water(grad[0], grad_old[0], self.mu_stop_w[0])[0] + self.rate(t_step)
        res[-1] = u_new[-1] - self.p_right/self.p_0

        return res

    # Метод Ньютона-Крылова для решения нелинейной системы
    def solve_nonlinear(self, u_old, s_old, t_step, dt, dx):
        """Решение системы уравнений с использованием метода Ньютона-Крылова."""
        u_new_guess = np.copy(u_old)  # начальное предположение
        # u_new = newton_krylov(lambda u_new: self.residual(u_new, u_old, s_old, t_step, dt, dx), u_new_guess,x_rtol=1e-4)
        # u_new = least_squares(lambda u_new: self.residual(u_new, u_old, s_old, t_step, dt, dx), u_new_guess, method='lm').x
        u_new = root(lambda u_new: self.residual(u_new, u_old, s_old, t_step, dt, dx), u_new_guess, method='lm').x
        return u_new

    # Параметры фракционного потока
    def fw(self, s, p, grad, grad_old, mu_w):
        """ Функция фракционного потока воды """
        grad = self.grad(p, self.dx)

        return self.lam_w(s, p, grad, grad_old, mu_w) * grad


    def exact(self, t):
        Swi0 = self.s_0
        self.mu_water0 = self.mu_o
        def f(sat):
            return k_water(sat)/(k_water(sat)+self.mu_water0/self.mu_o*k_oil(sat))

        def df(sat):
            return (dk_water(sat)*(k_water(sat)+self.mu_water0/self.mu_o*k_oil(sat))-k_water(sat)*(dk_water(sat)+self.mu_water0/self.mu_o*dk_oil(sat)))/(k_water(sat)+self.mu_water0/self.mu_o*k_oil(sat))**2

        def opt(sat):
            return f(sat)-f(Swi0)-df(sat)*(sat-Swi0)

        Sc = fsolve(opt, 0.5)[0]
        Xc = self.Q/self.m0*(f(Sc)-f(Swi0))/(Sc-Swi0)*t
        Sx1 = np.zeros(200)
        X1 = np.zeros(200)
        Sx2 = np.zeros(200)
        X2 = np.zeros(200)
        Sx1[0] = 1-Sor
        Sx2[0] = Swi0
        X2[0] = Xc
        for i in range(1,200):
            Sx1[i] = Sx1[i-1]-(1-Sor-Sc)/199 
            Sx2[i] = Swi0
            X2[i] = X2[i-1] + (self.L+1e-5-Xc)/199
        for i in range(1,200):
            X1[i] = self.Q/self.m0 * df(Sx1[i]) * t
        X3 = np.array([Xc, Xc])
        Sx3 = np.array([Sc, Swi0])
        return np.concatenate((X1,X3,X2)), np.concatenate((Sx1,Sx3,Sx2))


    def solve(self):
        p, s = self.solution_init()
        s_buckley = np.zeros_like(s)
        s_buckley[0,:] = s[0,:]
        mu_oil_init = np.zeros(self.nx, dtype=np.float64)+self.mu_h
        mu_water_init = np.zeros(self.nx, dtype=np.float64)+self.mu_h*1.5
        self.mu_oil_arr.append(mu_oil_init)
        self.mu_water_arr.append(mu_water_init)
        self.grad_p.append(self.grad(p[0,:], self.dx))
        self.grad_mu_o.append(self.grad(p[0,:], self.dx))
        self.grad_mu_w.append(self.grad(p[0,:], self.dx))
        # Решение системы с использованием Ньютона-Крылова
        for t, t_step in tqdm(enumerate(self.time[:-1]), total=self.time.shape[0], desc="Processing"):
            # решение уравнения для давления
            p_old = p[t, :].copy()
            s_old = s[t, :].copy()
            p_new = self.solve_nonlinear(p_old, s_old, t_step, self.dt, self.dx)
            p[t+1, :] = p_new.copy()
            grad_new = np.gradient(p_new, self.dx)
            grad_old = np.gradient(p_old, self.dx)
            # решение уравнения для насыщенности
            fw_val = self.fw(s_old, p_new, grad_new, grad_old, self.mu_stop_w)
            s_new = np.zeros_like(s[t,:], dtype=np.float64)
            s_new[1:-1] = s_old[1:-1] + self.dt*self.p_0*self.T/(self.rho_w(p_new[1:-1])*self.m(p_new[1:-1])*self.dx*self.L**2) * (fw_val[1:-1] - fw_val[:-2]) - s_old[1:-1]*self.p_0*(self.beta_r/self.m(p_new[1:-1]) + self.beta_w*self.rho_w0/self.rho_w(p_new[1:-1]))*(p_new[1:-1]-p_old[1:-1])
            # s_new[1:-1] = np.where(s_new[1:-1]>1, 1, s_new[1:-1])
            # s_new[1:-1] = np.where(s_new[1:-1]<0, 0, s_new[1:-1])
            s_new[0] = self.s_left
            s_new[-1] = s_new[-2]
            s[t+1,:] = s_new.copy()

            mu_oil_new, res_o = self.mu_oil(grad_new, self.grad_mu_o[-1], self.mu_stop_o)
            mu_water_new, res_w = self.mu_water(grad_new, self.grad_mu_w[-1], self.mu_stop_w)
            self.mu_stop_o = res_o
            self.mu_stop_w = res_w
            self.grad_mu_o.append(res_o*self.L/self.p_0)
            self.grad_mu_w.append(res_w*self.L/self.p_0)
            self.grad_p.append(grad_new)
            self.mu_oil_arr.append(mu_oil_new)
            self.mu_water_arr.append(mu_water_new)
            x_b, s_b = self.exact(t_step*self.T)
            x_b = x_b/self.L
            func = interp1d(x_b, s_b, kind='nearest')
            s_b = func(self.x)
            s_buckley[t+1, :] = s_b
        self.mu_oil_arr = np.array(self.mu_oil_arr)
        self.mu_water_arr = np.array(self.mu_water_arr)
        self.grad_p = np.array(self.grad_p)
        self.grad_mu_o = np.array(self.grad_mu_o)
        self.grad_mu_w = np.array(self.grad_mu_w)
        return p, s, x_b, s_buckley

In [ ]:
Square = (3.14*(3.5e-2)**2)/4

solver = TwoPhase(mu_h = 4.62e-3*2,
                mu_o = 4.62e-3,
                glad = 2.e-5,
                G = 9e6,
                k = 245.e-15,
                Q = 5.e-6/(Square*60),
                m0 = 0.2523,
                beta_r = 1e-10,
                rho_w0 = 990.,
                beta_w = 5e-10, # посмотреть в таблице
                rho_o0 = 800.,
                beta_o = 5e-9, # посмотреть в таблице
                t_0 = 0,
                T = 800,
                nt = 150*20,
                tt = 0.1,
                pow_n = 2,
                x_0 = 0,
                L = 6.1e-2,
                S = 0.3,
                nx = 50,
                p_0 = 1e+5,
                p_right=1e+5,
                s_0 = Swi,
                s_left = (1-Sor),
                s_right = 0.,
                mu_type_o = "mu",
                mu_type_w="mu_run")

p, s, x_b, s_b = solver.solve()

G = -np.ones_like(solver.grad_p)*solver.G

In [ ]:
solution_plot([[solver.x, p],
               [solver.x, s],
               [solver.x, s_b],
               [solver.x, solver.mu_oil_arr],
               [solver.x, solver.mu_water_arr],
               [solver.x, abs(G/solver.p_0*solver.L)],
               [solver.x, abs(G)/1.5/solver.p_0*solver.L],
               [solver.x, abs(solver.grad_p)],
               [solver.x, -solver.k*k_water(s)/solver.mu_water_arr*solver.grad_p],
               [solver.x, -solver.k*k_oil(s)/solver.mu_oil_arr*solver.grad_p]],
               titles=("Pressure (p) vs x", "Saturation (s) vs x", "mu_oil", "grad(p)", 'w_water_oil'),
               idx=[[1,1], [1,2], [1,2], [2,1], [2,1], [2,2], [2,2], [2,2], [3,1], [3,1]],
               nt=solver.nt,
               height=1000,
               width=1200,
               fraction=0.15)


In [ ]:
w_water = -solver.k*k_water(s)/solver.mu_water_arr*solver.grad_p
w_oil = -solver.k*k_oil(s)/solver.mu_oil_arr*solver.grad_p

Q_in = w_water[:,0]*solver.S
Qw_out = w_water[:,-1]*solver.S
Qo_out = w_oil[:,-1]*solver.S

Q_in_sum = [0]
Qw_out_sum = [0]
Qo_out_sum = [0]
for q_in_w, q_o, q_w in zip(Q_in, Qo_out, Qw_out):
    Q_in_sum.append(Q_in_sum[-1]+q_in_w)
    Qw_out_sum.append(Qw_out_sum[-1]+q_w)
    Qo_out_sum.append(Qo_out_sum[-1]+q_o)

V_in_sum = [q*(solver.S*solver.L*solver.m0) for q in Q_in_sum]

plt.plot(V_in_sum, Qw_out_sum, '--', label='water')
plt.plot(V_in_sum, Qo_out_sum, '--', label='oil')
plt.legend()
plt.show()